<a href="https://colab.research.google.com/github/flaviovdf/deep-ufmg/blob/main/listas/06-Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preâmbulo

Imports básicos


In [ ]:
! pip install torch==1.7.1
! pip install torchtext==0.8.1
! pip install torchvision==0.8.2

In [ ]:
! pip install unidecode

# Basic imports.
import os
import unidecode, re
import csv
import time
import random
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from sklearn import metrics
from sklearn.model_selection import train_test_split

from torchvision import models

from torchtext import data
from torchtext import datasets

import spacy
! python -m spacy download en
! python -m spacy download fr

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

cudnn.benchmark = True

SEED = 1234
torch.manual_seed(SEED)

In [ ]:
# Setting predefined arguments.
args = {
    'epoch_num': 100,       # Number of epochs.
    'lr': 1e-3,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 6,     # Number of workers on data loader.
    'batch_size': 10,     # Mini-batch size.
    'max_length': 50,    # Maximun length of predicted sentence
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

# Gerando Sequências


Dentre os tipos de problemas solucionáveis com modelos recorrentes, dois deles são baseados em geração de sequências: Os problemas One-to-Many,  e os Many-to-Many não sincronizados. <br>

Tipicamente os modelos de geração de sequências são baseados em arquiteturas **Encoder-Decoder**, onde a entrada é codificada para uma forma fixa, e então decodificada passo a passo em uma sequência.

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" width="600">


## Sequence-to-sequence models (Seq2Seq)

Modelos Sequence-to-Sequence (Seq2Seq) partem do mesmo princípio do Image Captioning, porém a entrada também é sequencial, de modo que a codificação também é realizada por um modelo recorrente.

A atividade de hoje é no contexto de Neural Machine Translation (NMT), cujo pipeline é representado de forma simplificada a seguir. Note que o idioma source (francês) necessita apenas do token de finalização de sentença, enquanto o idioma target precisa de ambos os inicializadores e os finalizadores (```<sos>```, ```<eos>```), visto que a entrada da rede precisa do token de inicialização, mas a saída, através da qual será calculada a loss, é produzida apenas com a finalização.

![](https://pytorch.org/tutorials/_images/seq2seq.png)

Imagem retirada do tutorial de NMT do Pytorch: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

Encontre modelos de linguagem pré-treinados e arquiteturas implementadas em: http://opennmt.net/


In [ ]:
# Baixando Dataset
!wget https://www.dropbox.com/s/gq36ksk347d36ln/translation_data.zip
!unzip translation_data.zip

In [ ]:
# Criando CSV de treino e teste para carregar com o TabularDataset

translation_path = 'data/eng-fra.txt'

samples = open(translation_path).read().split('\n')

# Write txt to csv
lines = (line.split("\t") for line in samples)
with open('translation_data.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('English', 'French'))
    writer.writerows(lines)

df = pd.read_csv('translation_data.csv')

# Reducing data (throwing out samples)
train, _ = train_test_split(df, test_size=0.95)

# Split train and test set
train, test = train_test_split(train, test_size=0.02)

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)


df = pd.read_csv('test.csv')
df.tail()

In [ ]:
# Preparação do dataset:
# Tokenização e inclusão dos tokens especiais (<eos>, <sos>, <pad>, <unk>)

def normalize_string(sentence):

  new_sentence = []
  for word in sentence:
    s = unidecode.unidecode(word.lower())
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)

    if s.strip() == '': continue

    new_sentence.append(s.strip())

  return new_sentence


TEXT_FR = data.Field(tokenize = 'spacy',
                     tokenizer_language='fr',
                     preprocessing = normalize_string,
                     include_lengths=True,
                     eos_token = "<eos>")
TEXT_EN = data.Field(tokenize = 'spacy',
                     tokenizer_language='en',
                     preprocessing = normalize_string,
                     include_lengths=True,
                     init_token = "<sos>",
                     eos_token = "<eos>")

fields = [('text_en', TEXT_EN), ('text_fr', TEXT_FR)]
train_data, test_data = data.TabularDataset.splits(
                                  path = '.',
                                  train = 'train.csv',
                                  test = 'test.csv',
                                  format = 'csv',
                                  fields = fields,
                                  skip_header = True)

for sample in train_data:
  print(sample.text_fr)
  print(sample.text_en)
  break

print(len(train_data), len(test_data))

In [ ]:
# Criando vocabulário
MAX_VOCAB_SIZE = 25000

TEXT_EN.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE)

TEXT_FR.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE)


# Instanciando bucket iterator
# Note que a ordenação é definida pelo
# comprimento do par ** (en, fr) **
# Precisaremos empacotar ambas as sequências
# para realizar o forward encapsulado.
train_iterator = data.BucketIterator(
    train_data,
    batch_size = 1,
    sort_key = lambda x:(len(x.text_fr), len(x.text_en)),
    sort_within_batch = True,
    device = args['device'])


test_iterator = data.BucketIterator(
    test_data,
    batch_size = 1,
    sort_within_batch = False,
    device = args['device'])

for k, batch in enumerate(train_iterator):
  text_fr, lengths_fr = batch.text_fr
  text_en, lengths_en = batch.text_en

  print(text_fr)
  print(text_en)
  print(lengths_fr)
  print(lengths_en)
  break


## Bahdanau Attention

Tutorial com gifs animados do Towards Data Science: https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3

Tutorial do Tensorflow usado como referência para a implementação: https://www.tensorflow.org/tutorials/text/nmt_with_attention

![](https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg)

### Passo a passo
<img width=650 src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg">

<img width=650 src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg">

Pseudo-código:

* `score = FC(tanh(FC(encoder_outputs) + FC(decoder_hidden)))`
* `attention weights = softmax(score, axis = 0)`.
   * Softmax é aplicada à dimensão 0 já que o score de alinhamento tem dimensionalidade `(seq_length, batch_size, hidden_size)`. `seq_length` é a quantidade de elementos da sequência. Já que buscamos associar um peso a cada elemento, devemos aplicar a softmax na dimensão correspondente.
* `context vector = sum(attention weights * EO, axis = 0)`.
* `embedding_output` = A entrada para o decoder (X) é passada por uma camada de embedding.
* `merged_vector = concat(embedding output, context vector)`. O vetor com a entrada e o `context_vector` são alimentados para a GRU do decoder.

In [ ]:
class BahdanauAttention(nn.Module):
  def __init__(self, encoder_size, decoder_size):

    super(BahdanauAttention, self).__init__()

    self.W1 = ### TODO
    self.W2 = ### TODO
    self.V  = ### TODO # values

  def forward(self, decoder_hidden, encoder_outputs): # query, keys

    # decoder_hidden   (1 x B x H)
    # encoder_outputs  (S x B x H)

    # score de alinhamento (S x B x 1)
    score = ### TODO

    # score como distribuição de probabilidades
    # softmax aplicada na dimensão da sequência
    # queremos um peso para cada elemento da sequência.
    attention_weights = ### TODO

    # aplica os scores (S x B x 1) no encoder_outputs (S x B x H)
    context_vector = ### TODO

    # Soma na dimensão da sequência -> (B x H)
    context_vector = ### TODO

    return context_vector, attention_weights.squeeze(-1)

### EncoderRNN

Implemente a classe **EncoderRNN** composta de um passo de representação de palavras e um passo de caracterização de sequência, ou seja, implemente as seguintes camadas:
*  Embedding: como não usaremos vetores pré-treinados, a dimensão de saída dessa camada é um hiperparâmetro livre. Sugestão de tamanho: ```100```. Sua entrada é definida pelo tamanho do dicionário do idioma source (nesse caso o francês).
*  Dropout: ```0.1``` <br><br>
*  GRU: Defina ```hidden_size = 128``` para o encoder

### DecoderRNN

Implemente a classe **DecoderRNN**. Novamente é necessário uma camada de representação de palavras, seguida de uma camada de caracterização de sequências. Além disso, o decoder também deve possuir uma camada Linear de classificação, que transformará a representação de cada timestep (saída da RNN) em uma predição da próxima palavra.

* Embedding: A entrada definida pelo vocabulário do idioma target (inglês), saída é um hiperparâmetro livre (sugestão: ```100```).
* Dropout: ```0.1``` <br><br>
* GRU: Seus hiperparâmetros são inferíveis a partir das outras informações. **Lembre-se que a inicialização do hidden state é dada pelo último hidden state do encoder** (veja na função train). <br><br>
* Linear: Parâmetros inferíveis pelas outras informações. Quantas classes tem a predição de palavras em inglês?
* LogSoftmax: ativação da classificação.

No decoder, **implemente ambos os forward** para treinamento (encapsulado em batches) e para inferência (loop explícito sem targets).

<img src="https://drive.google.com/uc?export=view&id=1j8aLVymyvhGtM0lpfON0aDyPvUf4700U" width="850">

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1, num_layers=1):
        super(EncoderRNN, self).__init__()

        ### TODO

    def forward(self, inputs, lengths):

        ### TODO

        ## Por favor, mantenha o retorno dessa forma
        return outputs, hidden[-1:]

    def initHidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).to(args['device'])


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length):

        ### TODO arquitetura do decoder
        ## GRU's input is [context, embed] (2*hidden_size)


    def forward(self, hidden, encoder_outputs, target, lengths):

        # token de input inicial
        token = torch.tensor(TEXT_EN.vocab.stoi["<sos>"]).unsqueeze(0).to(args['device'])

        outputs = []
        att_list = []
        for i in range(self.max_length):

          # Build context vector with the attention layer
          ### TODO

          # concatenate embedded representation with context vector
          ### TODO

          # recurrent forward and inference
          ### TODO

          topv, topi = output.topk(1)
          token = topi.squeeze(0).detach()  # detach from history as input

          outputs.append(output)
          if token == TEXT_EN.vocab.stoi["<eos>"]:
            break

        return torch.stack(outputs), torch.stack(att_list)


vocab_size_en = len(TEXT_EN.vocab)
hidden_size   = 128

vocab_size_fr = len(TEXT_FR.vocab)
max_length    = args['max_length']

encoder = EncoderRNN(vocab_size_fr, hidden_size).to(args['device'])
decoder = DecoderRNN(hidden_size, vocab_size_en, max_length).to(args['device'])

print(encoder)
print(decoder)

In [ ]:
# Setting optimizer.
encoder_optimizer = optim.Adam(encoder.parameters(),
                       lr=args['lr'],
                       weight_decay=args['weight_decay'],
                       betas=(args['momentum'], 0.999))

decoder_optimizer = optim.Adam(decoder.parameters(),
                       lr=args['lr'],
                       weight_decay=args['weight_decay'],
                       betas=(args['momentum'], 0.999))

# Setting loss.
criterion = nn.NLLLoss().to(args['device'])


In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy().squeeze(-1), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def train(train_loader, criterion, epoch):

    tic = time.time()

    # Setting network for training mode.
    encoder.train()
    decoder.train()

    # Lists for losses and metrics.
    train_loss = []

    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images, labels and paths for batch.
        text, text_lengths = batch_data.text_fr
        labs, labs_lengths = batch_data.text_en

        # Ignorando batches não ordenados para acelerar o treinamento
        if sorted(labs_lengths, reverse=True) != list(labs_lengths.data):
          continue

        # Clears the gradients of optimizer.
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Forwarding.
        enc, enc_hidden = encoder(text, text_lengths.cpu())
        outs, att = decoder(enc_hidden, enc, labs[:-1], labs_lengths.cpu()-1)
        minlen = len(outs)

        # Computing loss.
        loss = 0.
        for k in range(minlen):
          loss += criterion(outs[k], labs[k+1])
        loss = loss.mean()

        # Computing backpropagation.
        loss.backward()

        # Weight update
        encoder_optimizer.step()
        decoder_optimizer.step()

        # Updating lists.
        train_loss.append(loss.data.item())

        if (i+1) % 300 == 0:
          input = [TEXT_FR.vocab.itos[t] for t in text[:,0]]
          labels = [TEXT_EN.vocab.itos[t] for t in labs[:,0]]
          output = [TEXT_EN.vocab.itos[np.argmax(t.cpu().data)] for t in outs[:,0]]
          print(f'Input:{input}\nLabel:{labels}\nOutput:{output}',  )

          epoch_loss_ = np.asarray(train_loss)

          print('--------------------------------------------------------------------')
          print('[epoch %d iter %d], [train loss %.4f +/- %.4f], [epoch elapsed time %.2f]' % (
              epoch, i+1, epoch_loss_.mean(), epoch_loss_.std(), (time.time() - tic)))
          print('--------------------------------------------------------------------')

          showAttention(input, output, att)

    toc = time.time()

    train_loss = np.asarray(train_loss)

    # Printing training epoch loss and metrics.
    print('--------------------------------------------------------------------')
    print('[epoch %d], [train loss %.4f +/- %.4f], [training time %.2f]' % (
        epoch, train_loss.mean(), train_loss.std(), (toc - tic)))
    print('--------------------------------------------------------------------')

def test(test_loader, criterion, epoch):

    tic = time.time()

    # Setting network for evaluation mode (not computing gradients).
    encoder.eval()
    decoder.eval()

    # Lists for losses and metrics.
    test_loss = []

    print('********************************************************************')
    # Iterating over batches.
    for i, batch_data in enumerate(test_loader):

        # Obtaining images, labels and paths for batch.
        text, text_lengths = batch_data.text_fr
        labs, labs_lengths = batch_data.text_en


        # Forwarding.
        enc, enc_hidden  = encoder(text, text_lengths.cpu())
        outs, att = decoder(enc_hidden, enc)

        if i < 2:
          input = [TEXT_FR.vocab.itos[t] for t in text[:,0]]
          labels = [TEXT_EN.vocab.itos[t] for t in labs[:,0]]
          output = [TEXT_EN.vocab.itos[np.argmax(t.cpu().data)] for t in outs[:,0]]
          print(f'Input:{input}\nLabel:{labels}\nOutput:{output}',  )

          showAttention(input, output, att)

        # Computing approximate loss
        labs = labs[1:]
        minlen = min(len(labs), len(outs))

        loss = 0.
        for k in range(minlen):
          loss += criterion(outs[k], labs[k])
        loss = loss.mean()

        # Updating lists.
        test_loss.append(loss.data.item())

    toc = time.time()

    test_loss = np.asarray(test_loss)

    # Printing training epoch loss and metrics.

    print('[epoch %d], [test loss %.4f +/- %.4f], [testing time %.2f]' % (
        epoch, test_loss.mean(), test_loss.std(), (toc - tic)))
    print('********************************************************************')


In [ ]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train(train_iterator, criterion, epoch)

    # Computing test loss and metrics.
    test(test_iterator, criterion, epoch)

